In [4]:
# Chargement des données brutes
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import joblib



In [5]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

# On garde uniquement les lignes du train qui ont un score `sii`
train = train.dropna(subset=["sii"]).copy()
train["sii"] = train["sii"].astype(int)

In [6]:
# Colonnes numériques
num_cols = [
    "Basic_Demos-Age",
    "Physical-BMI",
    "PreInt_EduHx-computerinternet_hoursday",
    "SDS-SDS_Total_T"
]

# Colonnes catégorielles simples
cat_cols = [
    "Basic_Demos-Sex",                 # 0 = Garçon, 1 = Fille
    "Basic_Demos-Enroll_Season",       # Saison d'inscription
    "Physical-Season"                  # Saison des mesures physiques
]

# Sélection des colonnes + la cible
features = num_cols + cat_cols
target = "sii"

# ⚙️ Création des X, y
X = train[features].copy()
y = train[target]
X_test = test[features].copy()


In [7]:
print("🔍 NaN dans X :\n", X.isnull().sum())

🔍 NaN dans X :
 Basic_Demos-Age                             0
Physical-BMI                              209
PreInt_EduHx-computerinternet_hoursday     82
SDS-SDS_Total_T                           211
Basic_Demos-Sex                             0
Basic_Demos-Enroll_Season                   0
Physical-Season                           141
dtype: int64


In [8]:
# Séparation numérique / catégoriel
num_cols = ["Basic_Demos-Age", "Physical-BMI", "PreInt_EduHx-computerinternet_hoursday", "SDS-SDS_Total_T"]
cat_cols = ["Basic_Demos-Sex", "Basic_Demos-Enroll_Season", "Physical-Season"]

# Imputation numérique (médiane)
num_imputer = SimpleImputer(strategy="median")
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

# Imputation catégorielle (mode)
cat_imputer = SimpleImputer(strategy="most_frequent")
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
X_test[cat_cols] = cat_imputer.transform(X_test[cat_cols])

# Vérification
print(" Vérif NaN après imputation :")
print(X.isnull().sum())

 Vérif NaN après imputation :
Basic_Demos-Age                           0
Physical-BMI                              0
PreInt_EduHx-computerinternet_hoursday    0
SDS-SDS_Total_T                           0
Basic_Demos-Sex                           0
Basic_Demos-Enroll_Season                 0
Physical-Season                           0
dtype: int64


In [9]:
# On encode chaque colonne catégorielle une par une
encoders = {}

for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    X_test[col] = le.transform(X_test[col])  # attention : le test doit suivre l’ordre du train
    encoders[col] = le  # on les garde si besoin de décoder plus tard

print("Encodage terminé.")
print(X[cat_cols].head())

Encodage terminé.
   Basic_Demos-Sex  Basic_Demos-Enroll_Season  Physical-Season
0                0                          0                0
1                0                          2                0
2                1                          2                0
3                0                          3                2
5                1                          1                2


In [10]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# 🌟 Modèle XGBoost de base
xgb = XGBClassifier(random_state=42, n_jobs=1)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 🔍 Grille d'hyperparamètres
param_grid = {
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200],
    'subsample': [0.7, 1.0]
}

# ⚖️ GridSearch avec scoring QWK
grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='roc_auc_ovr',  # approximation multiclasses
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Entraînement avec GridSearch
grid.fit(X, y)

# Meilleurs paramètres trouvés
print("Meilleurs paramètres XGBoost:", grid.best_params_)

# Modèle final avec les meilleurs paramètres
final_xgb = XGBClassifier(**grid.best_params_, random_state=42, n_jobs=1)
final_xgb.fit(X_train, y_train)

# 📈 Prédictions
y_pred = final_xgb.predict(X_val)

# 🧪 Évaluation
qwk = cohen_kappa_score(y_val, y_pred, weights='quadratic')
print(f"XGBoost - QWK Score : {qwk:.4f}")


Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:54:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Meilleurs paramètres XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
[16:54:48] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost - QWK Score : 0.3355


##  Conclusion – Modèle XGBoost

Dans cette étape, nous avons testé le modèle **XGBoost** avec une recherche d'hyperparamètres (GridSearch) :

- **Meilleurs paramètres** trouvés :
  - `learning_rate = 0.1`
  - `max_depth = 3`
  - `subsample = 0.7`
  - `n_estimators = 100`
- **QWK score** avec XGBoost : **0.3355**

Bien que XGBoost soit un modèle performant, **LightGBM reste légèrement au-dessus avec un score QWK de 0.3499**. 

Conclusion : 
- **XGBoost est une bonne alternative**, mais pour ce cas précis, **LightGBM est légèrement plus performant** après optimisation.



In [12]:
# Sauvegarder le modèle XGBoost
joblib.dump(final_xgb, '../models/model_XGBoost.pkl')

['../models/model_XGBoost.pkl']